##### Exercise 5.1

Consider the diagrams on the right in Figure  5.2. Why does the value function jump up for the last two rows in the rear? Why does it drop off for the whole last row on the left? Why are the frontmost values higher in the upper diagrams than in the lower?

Jumps up for last two rows in the rear since sticking for 20 and 21 in blackjack usually result in a win. It drops off for the last row on the left since if the dealer has an Ace, it's bad news for the player. Finally, the frontmost values are higher in the upper diagrams than in the lower since having a usable Ace makes it less likely that an Ace will make the player bust.

##### Exercise 5.2

The backup diagram for the Monte Carlo estimation of $Q^\pi$ is similar to the backup diagram for $V^\pi$, but the root is a state-action pair, not just a state. The diagram ends in a terminal state.

##### Some notes 
"Without a model (as we had in DP chapter 4)... state values alone are not sufficient. One must explicitly estimate the value of each action in order for the values to be useful in suggesting a policy. "

By model, the author means, having the transition probabilities and transition rewards available at hand.

" For policy evaluation to work for action values, we must assure continual exploration. One way to do this is by specifying that the first step of each episode starts at a state-action pair, and that every such pair has a nonzero probability of being selected as the start. This guarantees that all state-action pairs will be visited an infinite number of times in the limit of an infinite number of episodes. We call this the assumption of exploring starts."


Has the following been proved on Monte Carlo ES?
"Convergence to this optimal fixed point seems inevitable as the changes to the action-value function decrease over time, but has not yet been formally proved. In our opinion, this is one of the most fundamental open questions in reinforcement learning."

##### Questions

**Question**: In 5.6, in the Figure 5.7, (c), the update to w, why is there a 1 in the numerator? I thought from 5.5 that the numerator should be $\pi(s_k,a_k)$. I guess the numerator would be 1 if the probability of obtaining the sequence given $\pi$ was 1, if $\pi$ were deterministic like the greedy policy.

And why are we taking $\tau$ to be the latest time at which the actions are not greedy? Is it because our estimate for $Q^\pi$ only improves for nongreedy actions as stated in the section?

**Question**: In 5.4, the conditions for the policy improvement theorem require optimal substructure right? So even though Monte Carlo is more robust to violations of the Markov Property since it doesn't bootstrap $V$ or $Q$, we are still assuming that greedy updates in GPI (Generalized Policy Improvement) will allow us to arrive at the optimal action-value functions due to the Markov Property?

##### Exercise 5.4

##### Exercise 5.5

##### Exercise 5.6

##### Exercise 5.7